In [95]:
import tweepy
import datetime
import pandas as pd

conskeys = "dEj9PuikWT7amd5Ud5k79jurB"
consecret ="5nGkmWffNKIepE5bKjzj6RNLn0RTjjjLgxeVFp19ZLFwcOP40H"
acctok = "895015384507498496-drN0ZC4fAcKZWG0uk4g1ACQvILaUECZ"
acctoksecret = "rcpKoy7OnipvbnblpPwVamPVvJKoQGt5Ux88sUz1tn0Oj"

auth = tweepy.OAuthHandler(conskeys, consecret)
auth.set_access_token(acctok, acctoksecret)

api = tweepy.API(auth)

hashtag = "#MODI OR #Narendra OR #PM OR #PrimeMinister"
since = datetime.datetime(2017, 1, 1)
until = datetime.datetime(2018, 1, 1)
max_tweets = 10  # Work this program twice for the two queries - go one by one and change to set 2 after 10k done
tpq = 5  # Tweets per query1
max_id = -1  # Setting a max_id to remove redundancy in case any of the queries raise an exception
count = 0

texts = []

for tweet in tweepy.Cursor(api.search, q=hashtag, count=100,since='2018-03-09',until='2018-03-17', lang='en', include_entities=False,tweet_mode='extended').items(max_tweets):
    try:
        tweets = (tweet._json.get('retweeted_status').get('full_text'))
    except AttributeError:
        tweets = (tweet._json.get('full_text'))
    texts.append(tweets)
    
df = pd.DataFrame(texts, columns=['text'])

In [96]:
import numpy as np 
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.text as Text
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


In [134]:
def filterData(data):
    data['text'] = data['text'].apply(lambda x: x.lower())
    data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

    max_fatures = 20
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(data['text'].values)
    X = tokenizer.texts_to_sequences(data['text'].values)
    # print(tokenizer.word_counts)
    X = pad_sequences(X)
    return X


In [149]:
pad_sequences?

In [136]:
X_test = filterData(df)


In [137]:
X_test

array([[19,  8, 11,  8, 12,  1,  1,  3,  4,  3,  5,  5,  9, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 10,  5,  2,  1,  1,  4],
       [ 0,  0,  0,  0, 13,  9, 12,  8, 14,  9,  8,  6, 12,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 15,  4, 16,  5,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 17,  3,  7, 15,  7,  6],
       [ 0,  0,  0,  0,  1,  3,  4,  4,  2, 10,  3,  1, 18, 15],
       [ 0,  0,  0, 14, 16, 18,  2, 13,  1, 17,  2,  7, 10, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  5],
       [ 0,  0,  0, 14, 16, 18,  2, 13,  1, 17,  2,  7, 10, 11]])

In [138]:
X = pd.read_csv('Sentiment.csv')
X.columns

Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [139]:
X = X.loc[:, ['sentiment', 'text']]
X_train = X.drop('sentiment', axis=1)
y_train = X.drop('text', axis=1)

In [140]:
y_train

,sentiment
0,Neutral
1,Positive
2,Neutral
3,Positive
4,Positive
5,Positive
6,Negative
7,Neutral
8,Negative
9,Negative


In [141]:
X_train = filterData(X_train)

In [142]:
X_train.shape

(13871, 14)

In [143]:
y_train = pd.get_dummies(y_train)

In [144]:
y_train

,sentiment_Negative,sentiment_Neutral,sentiment_Positive
0,0,1,0
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1
5,0,0,1
6,1,0,0
7,0,1,0
8,1,0,0
9,1,0,0


In [145]:

embed_dim = 128
lstm_out = 196
max_features = 2000
model = Sequential()

model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

C:\Users\mittr\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  import sys
C:\Users\mittr\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 14, 128)           256000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [146]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
13871/13871 [==============================] - 37s 3ms/step - loss: 0.9097 - acc: 0.6122
Epoch 2/2
13871/13871 [==============================] - 35s 3ms/step - loss: 0.8868 - acc: 0.6163


In [147]:
X_test.shape

(10, 14)

In [148]:
np.argmax(model.predict(X_test), axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)